# Toolkit Showcase: Prepare and run a protein-ligand simulation

<br />
<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>

    The simplest way to install dependencies is to install the examples package:
    
    conda install -c conda-forge openff-toolkit-examples
    
    This example will need access to a GROMACS install in addition to the above dependencies. Your existing GROMACS installed can be used, or you can install it from Bioconda:
    
    conda install -c bioconda gromacs
    
    You can also install all the depencies using the provided environment.yaml:
    
    conda env update --file ../environment.yaml 
    
    You may also need to restart this notebook's kernel after you make these changes (Kernel -> Restart)
</details>


In [1]:
%%html
<!--" Stylesheet for improved readability "-->
<style>
p, ul, ol, h1, h2, h3, h4, h5, h6 {
    max-width: 50rem;    
}
</style>

In [2]:
# Imports from the Python standard library
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

# Imports from dependencies
from simtk import openmm, unit
import parmed as pmd
import numpy as np
import mdtraj as mdt
import nglview

# Imports from the toolkit
import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.topology import Molecule, Topology

# Imports from local files
from utils import find_clashing_water, minimize_and_visualize

_(The OpenEye loading warning is expected -- The toolkit is informing us that OETK is unavailable, but it will safely fall back to using RDKit and AmberTools for the same functionality)_

## Introducing the main cast

Merck [provides data](https://github.com/MCompChem/fep-benchmark) to benchmark Free Energy Perturbation (FEP) procedures. We'll use structures from this dataset for this showcase:

https://github.com/MCompChem/fep-benchmark

This example is pre-packaged with one protein-ligand complex from the above repository, however you should be able to download other complexes and run them as well using this workflow. CHEMBL1078774, our ligand of choice, is an inhibitor of the mitotic functions of kinesin-5, a motor protein involved in cell division. 

The ligand and protein structures are already prepared for simulation:

- Their co-ordinates are super-imposable
- Hydrogens added to protein and crystallographic waters
- N-methyl and acetyl terminal caps on the protein to prevent unphysical charges
- Missing atoms are replaced

<br />
<details>
    <summary><small>▼ Click here for the shell commands we used to download the protein-ligand complex</small></summary>

    # Clone the repository
    git clone https://github.com/MCompChem/fep-benchmark.git
    # Take the first ligand from the eg5 benchmark
    head -n119 fep-benchmark/eg5/ligands.sdf > chembl_1078774.sdf
    # Take the prepared protein structure
    cp fep-benchmark/eg5/3l9h_prepared.pdb .
    
</details>

In [3]:
receptor_path: str = "3l9h_prepared.pdb"
ligand_path: str = "chembl_1078774.sdf"

We can visualize each structure using the [NGLView](https://github.com/nglviewer/nglview) widget. These visualizations are interactive; rotate by dragging the left mouse button, pan with the right mouse button, and zoom with the scroll wheel. You can also mouse over an atom to see its details, and click an atom to center the view on it. When you mouse over the widget, a full screen button will appear in its top right corner.

In [4]:
view: nglview.NGLWidget = nglview.show_file(ligand_path)
view

NGLWidget()

In [5]:
view: nglview.NGLWidget = nglview.show_file(receptor_path)
view

NGLWidget()

# The plan:

| Action | Software|
|--|--|
| Parameterize the ligand | OpenFF Toolkit
| Solvate the protein | OpenMM
| Parameterize the protein | OpenMM
| Combine the ligand and protein into a complex | ParmEd
| Remove waters that clash with the ligand | ParmEd/MDTraj
| Simulate the complex | OpenMM
| Visualize the simulation | NGLView

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
<!-- TODO: Explain in more detail what OpenMMForceFields, and decide whether to use this in the showcase -->
    ℹ️ Note that there's a new package `OpenMMForceFields` to replace much of this!
    <ul>
        <li> Home: <a href=https://github.com/openmm/openmmforcefields>https://github.com/openmm/openmmforcefields</a> </li>
        <li><code>conda install -c conda-forge -c omnia openmmforcefields</code></li>
        <li><a href=https://github.com/openforcefield/openforcefield/blob/master/examples/swap_amber_parameters/swap_existing_ligand_parameters_with_openmmforcefields.ipynb>Example notebook available</a></li>
    </ul>
</div>

## Parameterize the ligand (OFF Toolkit)

In this step, we'll produce parameters for our ligand from the unconstrained Parsley 1.3.0 force field. [Parsley](https://openforcefield.org/force-fields/force-fields/#parsley) is the first generation force field produced by the Open Force Field Initiative. Rather than using atom types like traditional biomolecular force fields, Parsley assigns parameters to a molecule with [fancy subgraph matching](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html). "Unconstrained" denotes that bonds involving hydrogen are treated as harmonics, like other bonds, rather than being constrained to a fixed length. This is appropriate when single-point energies must be as accurate as possible, though constraints allow a greater stable time step.

The Open Force Field Toolkit takes a molecular topology (the atoms in a molecule and their bonds) and the force field specification, and produces a `System` object that can be simulated with OpenMM or converted to inputs for other simulation packages. Note that, to produce a molecular topology, you need more than just the co-ordinates of the atoms; you also need their bonds, bond orders, and formal charges. As a result, `.sdf` files are used in this example; other file types are possible, but they must include this information.

In [6]:
# Load a molecule from a SDF file
ligand: Molecule = Molecule.from_file(ligand_path)
# Molecule loads both the co-ordinates of the atoms and their bond graph
ligand_positions: unit.Quantity = ligand.conformers[0]
ligand_topology: Topology = ligand.to_topology()

# Load the force field specification
off_forcefield: ForceField = ForceField("openff_unconstrained-1.3.0.offxml")

# Use the force field to produce an OpenMM system for the given topology
ligand_system: openmm.System = off_forcefield.create_openmm_system(ligand_topology)

_(takes ~100 seconds)_
### This is the only block in the first workflow that uses the Open Force Field Toolkit

In this workflow, the Toolkit is just responsible for combining a force field with a molecular topology. It is designed to check the input and give the user useful feedback if there seem to be errors or problems in the provided files. It also computes electric charges without user intervention. Computing charges is a process that can be confusing and error-prone, so we try to specify and simplify it as much as possible. Charges are computed efficiently with [OpenEye](https://www.eyesopen.com/) if is available; if it is not, the free toolkits [RDKit](https://www.rdkit.org/) and [AmberTools](https://ambermd.org/AmberTools.php) are used instead.


## Solvate and parameterize the protein (OpenMM)

The Parsley force field is designed for small molecule parameterization. Parameters for proteins and other polymers are coming, but for now we'll use Amber 99sb as our protein force field. The protein is already prepared with AMBER-compatible residue names, so this is simple.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that OpenMM and the Open Force Field Toolkit both have classes called `Topology` and `ForceField` that serve similar functions. Don't get them confused!
</div>


In [7]:
# Load protein and water force field parameters
omm_forcefield: openmm.app.ForceField = openmm.app.ForceField(
    "amber99sb.xml", "tip3p.xml"
)
# Load the kinesin-5 receptor structure and solvate it in 0.15 M NaCl solution
pdb: openmm.app.PDBFile = openmm.app.PDBFile(receptor_path)
modeller: openmm.app.Modeller = openmm.app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * unit.angstrom,
    ionicStrength=0.15 * unit.molar,
)
# Construct the OpenMM System from the solvated structure and the protein force field
protein_system: openmm.System = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME, 
    rigidWater=False # Necessary due to design differences between OpenFF/OpenMM and ParmEd
)

## Combine the parameterized ligand and the parameterized protein

[ParmEd](https://github.com/ParmEd/ParmEd) is able to read, write, combine, and edit topologies and structures from several different simulation programs, including CHARMM, OpenMM, Amber, and GROMACS. Here, we use it to combine our protein and ligand systems from OpenMM, remove clashing water molecules, and then write the combined system to OpenMM or GROMACS formats.

In [8]:
# Load the protein into a ParmEd Structure
protein_struct: pmd.Structure = pmd.openmm.load_topology(
    modeller.topology, protein_system, modeller.positions
)
# Load the ligand into a ParmEd Structure
ligand_struct: pmd.Structure = pmd.openmm.load_topology(
    ligand_topology.to_openmm(), ligand_system, ligand_positions
)

# ParmEd Structures override the "+" operator with a method that combines systems (!)
pmd_complex_struct: pmd.Structure = protein_struct + ligand_struct

# Assign periodic box vectors from the solvated receptor structure
pmd_complex_struct.box_vectors = modeller.topology.getPeriodicBoxVectors()

## Visualize the complex

NGLView supports a wide variety of [molecular visualization methods](https://nglviewer.org/ngl/api/manual/molecular-representations.html), as well as a VMD-like [atom selection language](https://nglviewer.org/ngl/api/manual/selection-language.html). This can be used to visualize complex systems like this one.

The widget consists of a minimally documented [Python library frontend](https://nglviewer.org/nglview/latest/api.html) and an extensively documented [JavaScript backend](https://nglviewer.org/ngl/api/manual/index.html). You'll need to refer to the documentation for both to do anything sophisticated, as the Python code delegates most of its options and functionality to the JS code.

In [9]:
# Create the widget. By default, proteins are shown as a cartoon and unrecognised ligands with a ball-and-stick model
view: nglview.NGLWidget = nglview.show_parmed(pmd_complex_struct)
    
# Add a licorice/stick representation for everything except the protein
view.add_licorice(selection="(not protein)")
# Make the ions clearer by drawing their vdW surfaces
view.add_surface(selection=":.NA or :.CL")

# Render the widget
view

NGLWidget()

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that there are waters clashing with the ligand, since the protein was solvated alone
</div>

## Remove waters that clash with the ligand (ParmEd/MDTraj)

In this step we use ParmEd to find and remove clashes from the combined system. Much of the work of this step is done by the `find_clashing_water` function, which can be found in the `utils.py` file accompanying this notebook.

In [10]:
clashes = find_clashing_water(pmd_complex_struct, "CHE", 0.15)

if len(clashes) != 0:
    clash_residues_str = ",".join([str(i) for i in clashes])
    print(f"Removing ligand-clashing water residues {clash_residues_str}")
    pmd_complex_struct.strip(f":{clash_residues_str}")
else:
    print("No ligand-water clashes to resolve")

view: nglview.NGLWidget = nglview.show_parmed(pmd_complex_struct)

view.add_licorice(selection="(not protein)")
view.add_surface(selection=":.NA or :.CL")
view

Removing ligand-clashing water residues 7475,7529


NGLWidget()

## Convert the combined system from ParmEd to our simulation engine of choice

Now that we have our system prepared in ParmEd, we can convert it to the engine we want to run our molecular dynamics simulation on. In this step, we'll use OpenMM; in the next, we'll use GROMACS.

In [11]:
# Create an OpenMM System from the ParmEd structure, adding in the appropriate parameters
system: openmm.System = pmd_complex_struct.createSystem(
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True, # Note that this is different to what we used to convert to ParmEd, due to differences in implementation between the two programs
)
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator: openmm.LangevinIntegrator = openmm.LangevinIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds
)

# Combine the topology from ParmEd with the new System and integrator
simulation: openmm.app.Simulation = openmm.app.Simulation(
    pmd_complex_struct.topology, system, integrator
)

# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
simulation.context.setPositions(
    pmd_complex_struct.positions + np.array([30, 30, 30]) * unit.angstrom
)

# Add a reporter to record the structure every 10 steps
nc_reporter: pmd.openmm.NetCDFReporter = pmd.openmm.NetCDFReporter("trajectory.nc", 10)
simulation.reporters.append(nc_reporter)

## Simulate the complex (OpenMM)
### Minimize the combined system

This will reduce any forces that are too large to integrate, such as from clashes or from disagreements between the crystal structure and force field.


In [12]:
simulation.minimizeEnergy()
minimized_coords: unit.Quantity = simulation.context.getState(
    getPositions=True
).getPositions()

_(Takes ~2 minutes on single core)_

### Run a short simulation

If this were anything more than a demonstration of the Toolkit, this example would need to include additional steps like equilibration. 

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Don't take this example as a description of a valid simulation protocol!
</div>

In [13]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

/home/joshmitchell/conda/envs/offdev/lib/python3.7/site-packages/parmed/amber/netcdffiles.py:409: UserWarning: Could not find netCDF4 module. Falling back on scipy implementation, which can significantly slow down simulations if used as a reporter
  warnings.warn('Could not find netCDF4 module. Falling back on '


_(Takes ~85 seconds on a single core, largely due to trajectory writing frequency)_

## While we wait, a few asides...

<!-- TODO: Get enough extra details to make sense of and expand on these -->

### Force Fields
* Reproducibility - User *must* see the name of what they're using
* Conda data packages - "Plugin" support for additional force fields (anybody can add!)
* Evolving together - Toolkit will support all functional forms in [OpenForceFields repo](https://github.com/openforcefield/openforcefields/)

<img src="img/openforcefields.png" alt="drawing" width="800"/>

<hr/>
    
### Charge generation
* Released FFs only use AM1-BCC, though different semiempirical methods and charge corrections are now available
* "Graph based" charges are coming in the near future -- Consistency and speed!
* Library charge support is available
    
    
<img src="img/xkcd_charge.png" alt="drawing" width="400"/>

<hr/>

### Current cheminformatics toolkit differences
* File formats
* Slight differences in partial charge
* Speed
* SMILES canonicalization
* Behavior stability
* Stereochemistry definition (Edge cases)

## Visualize the simulation (nglview)

NGLView can display trajectories, as well as single structures. Mouse over the widget to see the animation controls.

In [14]:
openmm.app.PDBFile.writeFile(
    pmd_complex_struct.topology, pmd_complex_struct.positions, open("system.pdb", "w")
)
mdt_traj = mdt.load("trajectory.nc", top="system.pdb")
print(mdt_traj)
import nglview

view = nglview.show_mdtraj(mdt_traj)
view

<mdtraj.Trajectory with 100 frames, 40097 atoms, 11915 residues, and unitcells>


NGLWidget(max_frame=99)

## Conclusions
* Toolkit parameterization requires *8 lines*, three of which are cheap hacks 
* Conda-installable, open source tools performed everything from basic system prep to simulation and visualization
* Using OpenMM, we never had to leave Python
* Using ParmEd, there was little additional work to running with GROMACS


<img src="img/dog_food.jpg" alt="drawing" width="350"/>

## What's next?

<!-- TODO: List some examples to continue on with after this one -->


# Appendix: What about GROMACS?

OpenMM makes it easy to run molecular simulations without leaving Python. The OpenFF toolkit currently exports directly to OpenMM, but no part of the parametrization process is exclusively supported by OpenMM. With ParmEd and other tools, the same systems can be run in other engines. Here we show how to use ParmEd to prepare and run the same workflow in GROMACS (_Thanks, Dennis Della Corte!_).

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Don't take this example as a description of a valid simulation protocol!
</div>

In [15]:
# Export GROMACS files.
pmd_complex_struct.save("system.top", overwrite=True)
pmd_complex_struct.save("system.gro", overwrite=True)

In [16]:
# TODO: Work out why net charge is  not zero - Seems too big for rounding errors
    # Reason is that the ligand has a charge of 0.004995 according to the topology
# TODO: Check if PARMED can produce position restraints, and return define = POSRES to equilibration MDPs
# TODO: Otherwise clean up the notes produced by GROMACS

# Configure GROMACS not to clutter up our directory with backups
%env GMX_MAXBACKUP=-1

# Energy minimise the coordinates
! gmx grompp -f minim.mdp -c system.gro -p system.top -o em.tpr -maxwarn 1
! gmx mdrun -v -deffnm em


env: GMX_MAXBACKUP=-1
                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meul

Step=   14, Dmax= 4.5e-02 nm, Epot= -4.83896e+05 Fmax= 7.80249e+03, atom= 2434
Step=   16, Dmax= 2.7e-02 nm, Epot= -4.87268e+05 Fmax= 3.24548e+04, atom= 2434
Step=   17, Dmax= 3.2e-02 nm, Epot= -4.90248e+05 Fmax= 1.66199e+04, atom= 2434
Step=   18, Dmax= 3.9e-02 nm, Epot= -4.90366e+05 Fmax= 4.21871e+04, atom= 2434
Step=   19, Dmax= 4.6e-02 nm, Epot= -4.93672e+05 Fmax= 2.78500e+04, atom= 2434
Step=   21, Dmax= 2.8e-02 nm, Epot= -4.96021e+05 Fmax= 1.40969e+04, atom= 2434
Step=   22, Dmax= 3.3e-02 nm, Epot= -4.96532e+05 Fmax= 3.64406e+04, atom= 2434
Step=   23, Dmax= 4.0e-02 nm, Epot= -4.99153e+05 Fmax= 2.41146e+04, atom= 2434
Step=   25, Dmax= 2.4e-02 nm, Epot= -5.01020e+05 Fmax= 1.22006e+04, atom= 2434
Step=   26, Dmax= 2.9e-02 nm, Epot= -5.01772e+05 Fmax= 3.16130e+04, atom= 2434
Step=   27, Dmax= 3.5e-02 nm, Epot= -5.03867e+05 Fmax= 2.08404e+04, atom= 2434
Step=   29, Dmax= 2.1e-02 nm, Epot= -5.05387e+05 Fmax= 1.06360e+04, atom= 2434
Step=   30, Dmax= 2.5e-02 nm, Epot= -5.06303e+05 Fma

Step=  146, Dmax= 1.2e-02 nm, Epot= -5.60856e+05 Fmax= 9.21520e+03, atom= 2434
Step=  147, Dmax= 1.4e-02 nm, Epot= -5.60965e+05 Fmax= 1.26318e+04, atom= 2434
Step=  148, Dmax= 1.7e-02 nm, Epot= -5.61113e+05 Fmax= 1.34371e+04, atom= 2434
Step=  149, Dmax= 2.1e-02 nm, Epot= -5.61126e+05 Fmax= 1.79486e+04, atom= 2434
Step=  150, Dmax= 2.5e-02 nm, Epot= -5.61217e+05 Fmax= 1.95291e+04, atom= 2434
Step=  152, Dmax= 1.5e-02 nm, Epot= -5.61821e+05 Fmax= 3.03872e+03, atom= 2434
Step=  154, Dmax= 8.9e-03 nm, Epot= -5.62061e+05 Fmax= 1.03377e+04, atom= 2434
Step=  155, Dmax= 1.1e-02 nm, Epot= -5.62292e+05 Fmax= 5.97419e+03, atom= 2434
Step=  156, Dmax= 1.3e-02 nm, Epot= -5.62355e+05 Fmax= 1.34557e+04, atom= 2434
Step=  157, Dmax= 1.5e-02 nm, Epot= -5.62612e+05 Fmax= 1.00073e+04, atom= 2434
Step=  159, Dmax= 9.2e-03 nm, Epot= -5.62844e+05 Fmax= 4.00085e+03, atom= 2434
Step=  160, Dmax= 1.1e-02 nm, Epot= -5.62968e+05 Fmax= 1.29957e+04, atom= 2434
Step=  161, Dmax= 1.3e-02 nm, Epot= -5.63256e+05 Fma

Step=  277, Dmax= 1.5e-02 nm, Epot= -5.78472e+05 Fmax= 6.56998e+03, atom= 1019
Step=  279, Dmax= 9.1e-03 nm, Epot= -5.78549e+05 Fmax= 7.41330e+03, atom= 1019
Step=  280, Dmax= 1.1e-02 nm, Epot= -5.78616e+05 Fmax= 9.41155e+03, atom= 1019
Step=  281, Dmax= 1.3e-02 nm, Epot= -5.78682e+05 Fmax= 1.07256e+04, atom= 1019
Step=  282, Dmax= 1.6e-02 nm, Epot= -5.78722e+05 Fmax= 1.34732e+04, atom= 1019
Step=  283, Dmax= 1.9e-02 nm, Epot= -5.78763e+05 Fmax= 1.54941e+04, atom= 1019
Step=  285, Dmax= 1.1e-02 nm, Epot= -5.78991e+05 Fmax= 1.92606e+03, atom= 1019
Step=  286, Dmax= 1.4e-02 nm, Epot= -5.79157e+05 Fmax= 1.88962e+04, atom= 1019
Step=  287, Dmax= 1.6e-02 nm, Epot= -5.79449e+05 Fmax= 6.26377e+03, atom= 1019
Step=  289, Dmax= 9.8e-03 nm, Epot= -5.79515e+05 Fmax= 8.77311e+03, atom= 1019
Step=  290, Dmax= 1.2e-02 nm, Epot= -5.79588e+05 Fmax= 9.31543e+03, atom= 1019
Step=  291, Dmax= 1.4e-02 nm, Epot= -5.79631e+05 Fmax= 1.23300e+04, atom= 1019
Step=  292, Dmax= 1.7e-02 nm, Epot= -5.79687e+05 Fma

Step=  409, Dmax= 9.7e-03 nm, Epot= -5.87956e+05 Fmax= 2.08971e+03, atom= 1019
Step=  410, Dmax= 1.2e-02 nm, Epot= -5.88049e+05 Fmax= 1.57845e+04, atom= 1019
Step=  411, Dmax= 1.4e-02 nm, Epot= -5.88209e+05 Fmax= 5.75021e+03, atom= 1019
Step=  413, Dmax= 8.4e-03 nm, Epot= -5.88260e+05 Fmax= 7.13429e+03, atom= 1019
Step=  414, Dmax= 1.0e-02 nm, Epot= -5.88306e+05 Fmax= 8.36349e+03, atom= 1019
Step=  415, Dmax= 1.2e-02 nm, Epot= -5.88347e+05 Fmax= 1.01904e+04, atom= 1019
Step=  416, Dmax= 1.5e-02 nm, Epot= -5.88380e+05 Fmax= 1.21023e+04, atom= 1019
Step=  417, Dmax= 1.7e-02 nm, Epot= -5.88401e+05 Fmax= 1.45917e+04, atom= 1019
Step=  418, Dmax= 2.1e-02 nm, Epot= -5.88407e+05 Fmax= 1.74479e+04, atom= 1019
Step=  420, Dmax= 1.3e-02 nm, Epot= -5.88608e+05 Fmax= 1.77069e+03, atom= 1019
Step=  421, Dmax= 1.5e-02 nm, Epot= -5.88696e+05 Fmax= 2.14216e+04, atom= 1019
Step=  422, Dmax= 1.8e-02 nm, Epot= -5.88964e+05 Fmax= 6.38122e+03, atom= 1019
Step=  424, Dmax= 1.1e-02 nm, Epot= -5.88995e+05 Fma

Step=  540, Dmax= 1.2e-02 nm, Epot= -5.95120e+05 Fmax= 9.72130e+03, atom= 1019
Step=  541, Dmax= 1.5e-02 nm, Epot= -5.95125e+05 Fmax= 1.31679e+04, atom= 1019
Step=  542, Dmax= 1.8e-02 nm, Epot= -5.95150e+05 Fmax= 1.43128e+04, atom= 1019
Step=  544, Dmax= 1.1e-02 nm, Epot= -5.95278e+05 Fmax= 2.16009e+03, atom= 1019
Step=  545, Dmax= 1.3e-02 nm, Epot= -5.95296e+05 Fmax= 1.77169e+04, atom= 1019
Step=  546, Dmax= 1.5e-02 nm, Epot= -5.95467e+05 Fmax= 6.05794e+03, atom= 1019
Step=  548, Dmax= 9.3e-03 nm, Epot= -5.95497e+05 Fmax= 8.21336e+03, atom= 1019
Step=  549, Dmax= 1.1e-02 nm, Epot= -5.95530e+05 Fmax= 8.88281e+03, atom= 1019
Step=  550, Dmax= 1.3e-02 nm, Epot= -5.95546e+05 Fmax= 1.16506e+04, atom= 1019
Step=  551, Dmax= 1.6e-02 nm, Epot= -5.95570e+05 Fmax= 1.29493e+04, atom= 1019
Step=  553, Dmax= 9.6e-03 nm, Epot= -5.95678e+05 Fmax= 1.83240e+03, atom= 1019
Step=  554, Dmax= 1.2e-02 nm, Epot= -5.95735e+05 Fmax= 1.58721e+04, atom= 1019
Step=  555, Dmax= 1.4e-02 nm, Epot= -5.95876e+05 Fma

Step=  672, Dmax= 8.0e-03 nm, Epot= -5.99997e+05 Fmax= 1.72450e+03, atom= 1019
Step=  673, Dmax= 9.6e-03 nm, Epot= -6.00048e+05 Fmax= 1.29263e+04, atom= 1019
Step=  674, Dmax= 1.1e-02 nm, Epot= -6.00144e+05 Fmax= 4.69151e+03, atom= 1019
Step=  676, Dmax= 6.9e-03 nm, Epot= -6.00172e+05 Fmax= 5.86187e+03, atom= 1019
Step=  677, Dmax= 8.3e-03 nm, Epot= -6.00200e+05 Fmax= 6.81751e+03, atom= 1019
Step=  678, Dmax= 9.9e-03 nm, Epot= -6.00223e+05 Fmax= 8.37870e+03, atom= 1019
Step=  679, Dmax= 1.2e-02 nm, Epot= -6.00243e+05 Fmax= 9.86711e+03, atom= 1019
Step=  680, Dmax= 1.4e-02 nm, Epot= -6.00255e+05 Fmax= 1.20018e+04, atom= 1019
Step=  681, Dmax= 1.7e-02 nm, Epot= -6.00258e+05 Fmax= 1.42393e+04, atom= 1019
Step=  683, Dmax= 1.0e-02 nm, Epot= -6.00378e+05 Fmax= 1.50396e+03, atom= 1019
Step=  684, Dmax= 1.2e-02 nm, Epot= -6.00422e+05 Fmax= 1.74469e+04, atom= 1019
Step=  685, Dmax= 1.5e-02 nm, Epot= -6.00580e+05 Fmax= 5.28993e+03, atom= 1019
Step=  687, Dmax= 8.9e-03 nm, Epot= -6.00599e+05 Fma

Step=  803, Dmax= 1.0e-02 nm, Epot= -6.04209e+05 Fmax= 8.66761e+03, atom= 1019
Step=  804, Dmax= 1.2e-02 nm, Epot= -6.04224e+05 Fmax= 1.00887e+04, atom= 1019
Step=  805, Dmax= 1.5e-02 nm, Epot= -6.04227e+05 Fmax= 1.24017e+04, atom= 1019
Step=  807, Dmax= 8.8e-03 nm, Epot= -6.04317e+05 Fmax= 1.10139e+03, atom= 1019
Step=  808, Dmax= 1.1e-02 nm, Epot= -6.04396e+05 Fmax= 1.50786e+04, atom= 1019
Step=  809, Dmax= 1.3e-02 nm, Epot= -6.04516e+05 Fmax= 4.41445e+03, atom= 1019
Step=  811, Dmax= 7.6e-03 nm, Epot= -6.04534e+05 Fmax= 7.25879e+03, atom= 1019
Step=  812, Dmax= 9.1e-03 nm, Epot= -6.04562e+05 Fmax= 6.75697e+03, atom= 1019
Step=  813, Dmax= 1.1e-02 nm, Epot= -6.04566e+05 Fmax= 1.00436e+04, atom= 1019
Step=  814, Dmax= 1.3e-02 nm, Epot= -6.04592e+05 Fmax= 1.01194e+04, atom= 1019
Step=  816, Dmax= 7.9e-03 nm, Epot= -6.04657e+05 Fmax= 1.97657e+03, atom= 1019
Step=  817, Dmax= 9.5e-03 nm, Epot= -6.04673e+05 Fmax= 1.25692e+04, atom= 1019
Step=  818, Dmax= 1.1e-02 nm, Epot= -6.04758e+05 Fma

Step=  934, Dmax= 1.3e-02 nm, Epot= -6.07756e+05 Fmax= 1.03809e+04, atom= 1019
Step=  936, Dmax= 7.8e-03 nm, Epot= -6.07821e+05 Fmax= 1.62475e+03, atom= 1019
Step=  937, Dmax= 9.4e-03 nm, Epot= -6.07840e+05 Fmax= 1.27178e+04, atom= 1019
Step=  938, Dmax= 1.1e-02 nm, Epot= -6.07925e+05 Fmax= 4.58856e+03, atom= 1019
Step=  940, Dmax= 6.8e-03 nm, Epot= -6.07944e+05 Fmax= 5.76524e+03, atom= 1019
Step=  941, Dmax= 8.1e-03 nm, Epot= -6.07962e+05 Fmax= 6.69929e+03, atom= 1019
Step=  942, Dmax= 9.7e-03 nm, Epot= -6.07975e+05 Fmax= 8.21424e+03, atom= 1019
Step=  943, Dmax= 1.2e-02 nm, Epot= -6.07986e+05 Fmax= 9.71874e+03, atom= 1019
Step=  944, Dmax= 1.4e-02 nm, Epot= -6.07987e+05 Fmax= 1.17494e+04, atom= 1019
Step=  946, Dmax= 8.4e-03 nm, Epot= -6.08068e+05 Fmax= 1.15752e+03, atom= 1019
Step=  947, Dmax= 1.0e-02 nm, Epot= -6.08116e+05 Fmax= 1.42026e+04, atom= 1019
Step=  948, Dmax= 1.2e-02 nm, Epot= -6.08220e+05 Fmax= 4.42004e+03, atom= 1019
Step=  950, Dmax= 7.3e-03 nm, Epot= -6.08234e+05 Fma

In [ ]:
# Run a short simulation
! gmx grompp -f sim.mdp -c em.gro -p system.top -o md.tpr -maxwarn 2
! gmx mdrun -v -deffnm md

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

_(Takes ~100 seconds)_

* Magic:
    * MDP files already prepared
    * `maxwarn 1` becuase of rounding errors with charges
* ParmEd *is* great, but *isn't* perfect, and we're actively working on bugfixes.
* We have philosophical differences about what constitutes "parameterization" 
    * Hbond constraints?
    * Electrostatics cutoffs?
* ParmEd is unable to process several OpenMM GBSA models


Yet to come - An OpenFF `System` class
* Could use a layer of indirection to make parameter optimization more efficient
* Will require resolving questions in the SMIRNOFF spec
    * How will the hierarchy of charge models be resolved?
    * How will `GBSA` and `Electrostatics` forces know to inherit the same charges?
    * Where will VirtualSites, which have both charge and vdW parameters, be defined?

In [ ]:
# TODO: PBC treatment?
mdt_traj = mdt.load("md.xtc", top="system.gro")
print(mdt_traj)

In [ ]:
import nglview

view = nglview.show_mdtraj(mdt_traj)
view